In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


# Explore data samples

In [2]:
# Data
trn_limit = -1
tst_limit = -1


In [3]:
import pandas as pd
train_df = pd.read_csv('../input/train.csv')[:trn_limit]
train_df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [4]:
import pandas as pd
test_df = pd.read_csv('../input/test.csv')[:tst_limit]
test_df.head()

,id,comment_text
0,7000000,Jeff Sessions is another one of Trump's Orwell...
1,7000001,I actually inspected the infrastructure on Gra...
2,7000002,No it won't . That's just wishful thinking on ...
3,7000003,Instead of wringing our hands and nibbling the...
4,7000004,how many of you commenters have garbage piled ...


# Data pre-processing

In [5]:
import numpy as np

In [6]:
trn_texts = list(train_df['comment_text'])
tst_texts = list(test_df['comment_text'])
texts = trn_texts + tst_texts

In [7]:
trn_texts[:10]

["This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",
 "Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!",
 'This is such an urgent design problem; kudos to you for taking it on. Very impressive!',
 "Is this something I'll be able to install on my site? When will you be releasing it?",
 'haha you guys are a bunch of losers.',
 'ur a sh*tty comment.',
 'hahahahahahahahhha suck it.',
 'FFFFUUUUUUUUUUUUUUU',
 'The ranchers seem motivated by mostly by greed; no one should have the right to allow their animals destroy public land.',
 "It was a great show. Not a combo I'd of expected to be good together but it was."]

In [8]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

Using TensorFlow backend.


In [9]:
del texts
del tst_texts
del test_df

In [10]:
binary_trn_texts = tokenizer.texts_to_sequences(trn_texts)

In [11]:
maxlen = max([len(s) for s in binary_trn_texts])
maxlen

317

In [12]:
from keras.preprocessing.sequence import pad_sequences

x_train = np.array(pad_sequences(binary_trn_texts, maxlen=maxlen, padding='post', truncating='post'))
x_train[:10]

array([[    18,      6,     34, ...,      0,      0,      0],
       [   406,     10,     18, ...,      0,      0,      0],
       [    18,      6,    149, ...,      0,      0,      0],
       ...,
       [195872,      0,      0, ...,      0,      0,      0],
       [     1,  12600,    393, ...,      0,      0,      0],
       [    12,     22,      5, ...,      0,      0,      0]], dtype=int32)

In [13]:
del binary_trn_texts

In [14]:
y_train = np.array((list((train_df['target']>0.5)*1)))
y_train[:10]

array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0])

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42, shuffle=True)

In [16]:
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(1624385, 317)
(180488, 317)
(1624385,)
(180488,)


# Model

In [17]:
model_name = 'bigru_model_1'

In [18]:
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout
from keras.layers import Embedding, TimeDistributed
from keras.layers import LSTM, GRU, Bidirectional
from keras.layers import Conv1D, MaxPooling1D

vocab_size = len(tokenizer.word_index)

# Model
# Embedding
embedding_size = 200

# LSTM
lstm_output_size = 100


'''
# Model 1: Simple LSTM, with Embedding mask_zero
model = Sequential()
model.add(Embedding(vocab_size+1, embedding_size, mask_zero=True, input_length=maxlen))
model.add(LSTM(lstm_output_size))
model.add(Dropout(0.2))
#model.add(Dense(500))
#model.add(Dropout(0.25))
#model.add(Dense(200))
#model.add(Dropout(0.25))
#model.add(Dense(100))
#model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
'''

'''
# Model 2: BiGRU
sentence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = Embedding(vocab_size+1, embedding_size, mask_zero=True)(sentence_input)
l_lstm = Bidirectional(GRU(lstm_output_size))(embedded_sequences)
l_lstm = Dense(lstm_output_size, activation='relu')(l_lstm)
output = Dense(1, activation='sigmoid')(l_lstm)
model = Model(sentence_input, output)
'''



# Model 3: BiGRU + distributed dense + GRU + Dense
sentence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = Embedding(vocab_size+1, embedding_size, mask_zero=True)(sentence_input)
l_lstm = Bidirectional(GRU(lstm_output_size, return_sequences=True))(embedded_sequences)
l_word_enc = TimeDistributed(Dense(lstm_output_size))(l_lstm)
l_lstm_2 = GRU(lstm_output_size)(l_word_enc) #Bidirectional
#l_dense = Dense(200, activation='relu')(l_word_enc)
l_dense = Dense(lstm_output_size, activation='relu')(l_lstm_2)
output = Dense(1, activation='sigmoid')(l_lstm_2)
model = Model(sentence_input, output)






model.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 317)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 317, 200)          82009200  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 317, 200)          180600    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 317, 100)          20100     
_________________________________________________________________
gru_2 (GRU)                  (None, 100)               60300     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 82,270,301
Trainable params: 82,270,301
Non-trainable pa

In [19]:
from keras.callbacks import ModelCheckpoint, Callback
#filepath = os.path.join(gdrive_path, 'jigsaw_' + model_name + '.h5')
checkpoint = ModelCheckpoint(filepath='./'+model_name+'.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')


In [20]:
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score, auc
class roc_callback(Callback):
    def __init__(self, x_val, y_val):
        
        self.x_val = x_val
        # y_val is continous, but according to the competetion, the thresh is at 0.5, so we can convert to binary
        # The *1 is to convert False/True into 0/1    
        self.y_val = np.array((y_val>0.5)*1)
        
    
    def on_train_begin(self, logs={}):
        return
 
    def on_train_end(self, logs={}):
        return
 
    def on_epoch_begin(self, epoch, logs={}):
        return
 
    def on_epoch_end(self, epoch, logs={}):        
        y_pred = self.model.predict(self.x_val)

        val_auc  = roc_auc_score(self.y_val, model.predict(x_val))

        
        print('Epoch: ' + str(epoch) + ' AUC: ' + str(val_auc))
        
        return
 
    def on_batch_begin(self, batch, logs={}):
        return
 
    def on_batch_end(self, batch, logs={}):
        return 

In [21]:
from keras import optimizers
# Optimizer
lr = 0.001
opt = optimizers.Adam(lr=lr)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [22]:
print('Train...')
#callbacks_lst = [checkpoint, roc_callback(x_val, y_val)]
callbacks_lst = [checkpoint]
# Training
batch_size = 1024
epochs = 2
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,          
          validation_data=(x_val, y_val),
          #validation_split=0.2,
          callbacks=callbacks_lst)

Train...
Instructions for updating:
Use tf.cast instead.
Train on 1624385 samples, validate on 180488 samples
Epoch 1/2
1624385/1624385 [==============================] - 2323s 1ms/step - loss: 0.1107 - acc: 0.9611 - val_loss: 0.0963 - val_acc: 0.9642

Epoch 00001: val_acc improved from -inf to 0.96421, saving model to ./bigru_model_1.h5
Epoch 2/2
1624385/1624385 [==============================] - 2300s 1ms/step - loss: 0.0827 - acc: 0.9677 - val_loss: 0.0972 - val_acc: 0.9647

Epoch 00002: val_acc improved from 0.96421 to 0.96473, saving model to ./bigru_model_1.h5


# Submission

In [23]:
test_df = pd.read_csv('../input/test.csv')
tst_texts = list(test_df['comment_text'])
binary_tst_texts = tokenizer.texts_to_sequences(tst_texts)
x_test = np.array(pad_sequences(binary_tst_texts, maxlen=maxlen))
y_test = model.predict(x_test)

In [24]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='id')
submission['prediction'] = y_test
submission.reset_index(drop=False, inplace=True)
submission.head()
submission.to_csv('submission.csv', index=False)